# Stock Sentiment Analysis using News Headlines

The data is from Kaggle and here we have to predict whether stock price will increase or decrease based on news headlines. Date column have the date, label is 1 or 0. 1 means the stock price will increase, 0 means it will remain same or decrease. All the other columns containt the top news headlines of the day.

In [3]:
import pandas as pd

In [4]:
data_path = "data/data.csv"

In [5]:
df = pd.read_csv(data_path, encoding="ISO-8859-1")

In [6]:
df.head()

Date  ...                         Top25
0  2000-01-03  ...            Recovering a title
1  2000-01-04  ...  Millennium bug fails to bite
2  2000-01-05  ...                  Useful links
3  2000-01-06  ...   Lessons of law's hard heart
4  2000-01-07  ...                    Gone aloft

[5 rows x 27 columns]

In [7]:
df.shape

(4101, 27)

In [8]:
train = df[df['Date'] < '20150101']
test = df[df['Date'] > '20141231']

In [9]:
print(train.shape)
print(test.shape)

(3975, 27)
(378, 27)


In [10]:
# removing punctuations
data = train.iloc[:, 2:27]
data.replace("[^a-zA-Z]", " ", regex=True, inplace=True)

# renaming column names
new_cols = [str(i) for i in range(25)]
data.columns = new_cols

In [11]:
data.head(5)

0  ...                            24
0  A  hindrance to operations   extracts from the...  ...            Recovering a title
1                                          Scorecard  ...  Millennium bug fails to bite
2                  Coventry caught on counter by Flo  ...                  Useful links
3                      Pilgrim knows how to progress  ...   Lessons of law s hard heart
4                               Hitches and Horlocks  ...                    Gone aloft

[5 rows x 25 columns]

In [12]:
# converting headlines to lowercase
for col in new_cols:
    data[col] = data[col].str.lower()

In [13]:
data.head()

0  ...                            24
0  a  hindrance to operations   extracts from the...  ...            recovering a title
1                                          scorecard  ...  millennium bug fails to bite
2                  coventry caught on counter by flo  ...                  useful links
3                      pilgrim knows how to progress  ...   lessons of law s hard heart
4                               hitches and horlocks  ...                    gone aloft

[5 rows x 25 columns]

In [14]:
# combining all the headlines to one paragraph
headlines = []
for row in range(0, len(data.index)):
    headlines.append(' '.join(str(x) for x in data.iloc[row, 0:25] ))

In [15]:
headlines[0]

'a  hindrance to operations   extracts from the leaked reports scorecard hughes  instant hit buoys blues jack gets his skates on at ice cold alex chaos as maracana builds up for united depleted leicester prevail as elliott spoils everton s party hungry spurs sense rich pickings gunners so wide of an easy target derby raise a glass to strupar s debut double southgate strikes  leeds pay the penalty hammers hand robson a youthful lesson saints party like it s      wear wolves have turned into lambs stump mike catches testy gough s taunt langer escapes to hit     flintoff injury piles on woe for england hunters threaten jospin with new battle of the somme kohl s successor drawn into scandal the difference between men and women sara denver  nurse turned solicitor diana s landmine crusade put tories in a panic yeltsin s resignation caught opposition flat footed russian roulette sold out recovering a title'

### Building the model

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

Let's write some utility functions

In [20]:
def build_model(X_train, y_train, X_test, y_test):

    models = {
        "rf": RandomForestClassifier(n_jobs=-1),
        "lr": LogisticRegression(),
        "mnb": MultinomialNB()
    }

    for model in models:
        model = models[model]
        model.fit(X_train, y_train)

        predictions = model.predict(X_test)
        
        print(f"Model: {model}")
        print(accuracy_score(y_test, predictions))
        print(confusion_matrix(y_test, predictions))
        print(classification_report(y_test, predictions))

        print("="*100)


    

### Bag of words
Let's first start with a simple bag of words model



In [21]:
# implement bag of words
countvector = CountVectorizer(ngram_range=(2,2))
X_train = countvector.fit_transform(headlines)

In [22]:
y_train = train['Label']

test_transform = []
for row in range(0, len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row, 2:27]))

X_test = countvector.transform(test_transform)
y_test = test['Label']

In [23]:
build_model(X_train, y_train, X_test, y_test)

Model: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)
0.8597883597883598
[[137  49]
 [  4 188]]
              precision    recall  f1-score   support

           0       0.97      0.74      0.84       186
           1       0.79      0.98      0.88       192

    accuracy                           0.86       378
   macro avg       0.88      0.86      0.86       378
weighted avg       0.88      0.86      0.86       378

Model: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=T

### TF-IDF

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X_train = tfidf.fit_transform(headlines).toarray()

In [25]:
test_transform = []
for row in range(0, len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row, 2:27]))

X_test = tfidf.transform(test_transform)

In [26]:
build_model(X_train, y_train, X_test, y_test)

Model: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)
0.8465608465608465
[[145  41]
 [ 17 175]]
              precision    recall  f1-score   support

           0       0.90      0.78      0.83       186
           1       0.81      0.91      0.86       192

    accuracy                           0.85       378
   macro avg       0.85      0.85      0.85       378
weighted avg       0.85      0.85      0.85       378

Model: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=T